In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz

In [38]:
relation = pd.read_csv("/Users/yyang/ucsb/robust04/qrels.rob04.txt", sep=" ",
                      names = ["id_left", "dummy", "id_right", "label"]).drop("dummy",axis=1)
relation['label'] = relation['label'].astype(float)
relation = relation.dropna(axis=0, how='any')
relation.id_left = relation.id_left.astype("str")
relation_id_left = relation.id_left.drop_duplicates()
relation.id_right = relation.id_right.astype("str")
relation_id_right = relation.id_right.drop_duplicates()
relation = relation.dropna(axis=0, how='any')

In [42]:
left = pd.read_csv("/Users/yyang/ucsb/robust04/query_trec45_no.txt", sep=":",
                  names = ["id_left", "text_left"])
left = left.dropna(axis=0, how='any')
left.id_left = left.id_left.astype("str")

In [44]:
doc_dict = dict()
with open("/Users/yyang/ucsb/robust04/trec45.doc.txt") as f:
    for line in f:
        info = line.split("\t")
        if len(info) == 4:
            text = info[2] + " ".join(info[3].split(" ")[:500])
        else:
            text = " ".join(info[2].split(" ")[:500])
        doc_dict[info[1]] = text

In [45]:
right = pd.DataFrame(list(doc_dict.items()),columns = ['id_right','text_right']) 
right = right.dropna(axis=0, how='any')
right.id_right = right.id_right.astype("str")
right_id_right = right.id_right.drop_duplicates()

In [48]:
train_id_left = pd.read_csv('../../../cedr/data/robust/f1.train.pairs', sep = "\t",
                         names = ["id_left", "id_right"]).id_left.astype("str").drop_duplicates()
vali_id_left = pd.read_csv('../../../cedr/data/robust/f1.valid.run', sep = " ",
                         names = ["id_left", "dummy1", "id_right", "rank", "score", "dummy2"]).id_left.astype("str").drop_duplicates()
test_id_left = pd.read_csv('../../../cedr/data/robust/f1.test.run', sep = " ",
                         names = ["id_left", "dummy1", "id_right", "rank", "score", "dummy2"]).id_left.astype("str").drop_duplicates()

In [58]:
relation_train = pd.merge(pd.merge(relation,train_id_left, on=["id_left"], how="inner"), 
                          right_id_right, how = "inner")[["id_left", "id_right", "label"]]
relation_vali = pd.merge(pd.merge(relation,vali_id_left, on=["id_left"], how="inner"), 
                          right_id_right, how = "inner")[["id_left", "id_right", "label"]]
relation_test = pd.merge(pd.merge(relation,test_id_left, on=["id_left"], how="inner"), 
                          right_id_right, how = "inner")[["id_left", "id_right", "label"]]

left_train = pd.merge(train_id_left, left, how="inner", on="id_left")[["id_left", "text_left"]]
left_train.set_index("id_left", inplace=True)
left_vali = pd.merge(vali_id_left, left, how="inner", on="id_left")[["id_left", "text_left"]]
left_vali.set_index("id_left", inplace=True)
left_test = pd.merge(test_id_left, left, how="inner", on="id_left")[["id_left", "text_left"]]
left_test.set_index("id_left", inplace=True)

right_train = pd.merge(relation_train, right, how="inner", on="id_right")[["id_right", "text_right"]].drop_duplicates()
right_train.set_index("id_right", inplace=True)
right_vali = pd.merge(relation_vali, right, how="inner", on="id_right")[["id_right", "text_right"]].drop_duplicates()
right_vali.set_index("id_right", inplace=True)
right_test = pd.merge(relation_test, right, how="inner", on="id_right")[["id_right", "text_right"]].drop_duplicates()
right_test.set_index("id_right", inplace=True)

In [65]:
right_train

,text_right
id_right,
FBIS3-10169,language spain article type bfn state of the n...
FBIS3-10243,language spain article type bfn from the natio...
FBIS3-10319,language spain article type bfn report by basi...
FBIS3-10397,language spain article type bfn text national ...
FBIS3-10491,language spain article type cso article by cam...
FBIS3-10635,language spain article type bfn report by alva...
FBIS3-10721,language spain article type bfn interview with...
FBIS3-10910,language spain article type bfn excerpt the in...
FBIS3-10937,language spain article type cso text amid the ...


In [62]:
right_train.text_right.drop_duplicates().shape

(93718,)

In [ ]:
print('data loading ...')
train_pack_raw = mz.DataPack(relation=relation_train,left=left_train,right=right_train)
dev_pack_raw = mz.DataPack(relation=relation_vali,left=left_vali,right=right_vali)
test_pack_raw = mz.DataPack(relation=relation_test,left=left_test,right=right_test)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')